# Field Line Detection on a Football Pitch: Multi-Algorithm Comparison

This notebook demonstrates a traditional image-processing pipeline for detecting field lines on a football pitch. In each step we provide multiple algorithm options:

1. **Crowd Masking:** Uses the fact that the pitch is predominantly green to mask out non-pitch areas.
   - Option A: HSV thresholding
   - Option B: LAB thresholding

2. **White Line Extraction:** Isolates the white markings from the pitch region.
   - Option A: Simple binary thresholding
   - Option B: Adaptive thresholding

3. **Edge Detection:** Extracts edges using:
   - Option A: Canny edge detector
   - Option B: Sobel operator

4. **Line Detection:** Detects line segments using:
   - Option A: Probabilistic Hough Transform (HoughLinesP)
   - Option B: Line Segment Detector (LSD)

At the end, the notebook iterates over all combinations (16 in total) and displays the final output images in a grid so that you can compare which combination works best for your data.

In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
import sys
from itertools import product

def show_image(img, title='Image', cmap=None):
    plt.figure(figsize=(10, 6))
    if cmap:
        plt.imshow(img, cmap=cmap)
    else:
        if len(img.shape) == 3:
            plt.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
        else:
            plt.imshow(img)
    plt.title(title)
    plt.axis('off')
    plt.show()

def debug_print(msg):
    print("[DEBUG] " + msg, file=sys.stderr)

debug_print("Libraries imported successfully.")

In [ ]:
# Load the input image
image_path = 'input.jpg'
image = cv2.imread(image_path)

if image is None:
    debug_print(f"Error: Could not load image '{image_path}'.")
    sys.exit(1)
else:
    debug_print(f"Image '{image_path}' loaded. Shape: {image.shape}")
    show_image(image, 'Original Image')

## (Optional) Individual Step Demonstrations

The following cells show each step separately with debugging output. You can run these cells to verify individual steps before comparing combinations.

### Step 1: Crowd Masking

We mask out non-pitch areas by leveraging the pitch's predominantly green color. Two methods are provided (choose between HSV and LAB).

In [ ]:
# Choose masking method: 'HSV' or 'LAB'
mask_method = 'HSV'  # Change to 'LAB' to try LAB thresholding
debug_print(f"Using {mask_method} thresholding for pitch masking.")

if mask_method == 'HSV':
    hsv = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
    debug_print("Converted image to HSV.")
    lower_green = np.array([25, 20, 20])
    upper_green = np.array([95, 255, 255])
    pitch_mask = cv2.inRange(hsv, lower_green, upper_green)
    debug_print("HSV mask created.")
elif mask_method == 'LAB':
    lab = cv2.cvtColor(image, cv2.COLOR_BGR2LAB)
    debug_print("Converted image to LAB.")
    lower_lab = np.array([0, 110, 0])
    upper_lab = np.array([255, 140, 255])
    pitch_mask = cv2.inRange(lab, lower_lab, upper_lab)
    debug_print("LAB mask created.")
else:
    debug_print("Invalid mask method selected. Defaulting to HSV.")
    hsv = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
    lower_green = np.array([25, 20, 20])
    upper_green = np.array([95, 255, 255])
    pitch_mask = cv2.inRange(hsv, lower_green, upper_green)

# Apply morphological operations to smooth the mask
kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (7, 7))
pitch_mask = cv2.morphologyEx(pitch_mask, cv2.MORPH_CLOSE, kernel, iterations=2)
pitch_mask = cv2.dilate(pitch_mask, kernel, iterations=1)
debug_print("Morphological operations applied to mask.")
show_image(pitch_mask, 'Pitch Mask', cmap='gray')

# Extract pitch region (mask out crowd)
pitch_only = cv2.bitwise_and(image, image, mask=pitch_mask)
debug_print("Extracted pitch region using the mask.")
show_image(pitch_only, 'Pitch Only (Crowd Masked Out)')

### Step 2: White Line Extraction

Two approaches are provided:

- **Simple Binary Thresholding**: A fixed threshold is used.
- **Adaptive Thresholding**: The threshold is computed locally.

In [ ]:
# Choose thresholding method: 'simple' or 'adaptive'
thresh_method = 'simple'  # Change to 'adaptive' to try adaptive thresholding
debug_print(f"Using {thresh_method} thresholding for white line extraction.")

gray = cv2.cvtColor(pitch_only, cv2.COLOR_BGR2GRAY)
show_image(gray, 'Grayscale Pitch Image', cmap='gray')

if thresh_method == 'simple':
    ret, thresh = cv2.threshold(gray, 200, 255, cv2.THRESH_BINARY)
    debug_print(f"Simple threshold applied. ret = {ret}")
elif thresh_method == 'adaptive':
    thresh = cv2.adaptiveThreshold(gray, 255, cv2.ADAPTIVE_THRESH_MEAN_C, cv2.THRESH_BINARY, blockSize=11, C=2)
    debug_print("Adaptive threshold applied.")
else:
    debug_print("Invalid threshold method selected. Defaulting to simple thresholding.")
    ret, thresh = cv2.threshold(gray, 200, 255, cv2.THRESH_BINARY)

show_image(thresh, 'Thresholded Image (White Lines Highlighted)', cmap='gray')

### Step 3: Edge Detection

Two options are provided:

- **Canny Edge Detector**
- **Sobel Operator**

In [ ]:
# Choose edge detection method: 'canny' or 'sobel'
edge_method = 'canny'  # Change to 'sobel' to try Sobel operator
debug_print(f"Using {edge_method} edge detection.")

if edge_method == 'canny':
    edges = cv2.Canny(thresh, 50, 150, apertureSize=3)
    debug_print("Canny edge detection applied.")
elif edge_method == 'sobel':
    sobelx = cv2.Sobel(thresh, cv2.CV_64F, 1, 0, ksize=3)
    sobely = cv2.Sobel(thresh, cv2.CV_64F, 0, 1, ksize=3)
    edges = cv2.magnitude(sobelx, sobely).astype(np.uint8)
    debug_print("Sobel edge detection applied.")
else:
    debug_print("Invalid edge method selected. Defaulting to Canny.")
    edges = cv2.Canny(thresh, 50, 150, apertureSize=3)

show_image(edges, 'Edge Map', cmap='gray')

### Step 4: Line Detection

Finally, two options are provided for detecting line segments:

- **HoughLinesP (Probabilistic Hough Transform)**
- **Line Segment Detector (LSD)**

In [ ]:
# Choose line detection method: 'hough' or 'lsd'
line_method = 'hough'  # Change to 'lsd' to try Line Segment Detector
debug_print(f"Using {line_method} for line detection.")

line_image = pitch_only.copy()

if line_method == 'hough':
    lines = cv2.HoughLinesP(edges, rho=1, theta=np.pi/180, threshold=50, minLineLength=50, maxLineGap=10)
    if lines is not None:
        debug_print(f"HoughLinesP detected {len(lines)} lines.")
        for line in lines:
            x1, y1, x2, y2 = line[0]
            cv2.line(line_image, (x1, y1), (x2, y2), (0, 0, 255), 2)
    else:
        debug_print("No lines detected using HoughLinesP.")
elif line_method == 'lsd':
    lsd = cv2.createLineSegmentDetector(0)
    lines_lsd = lsd.detect(edges)[0]
    if lines_lsd is not None:
        debug_print(f"LSD detected {len(lines_lsd)} line segments.")
        for line in lines_lsd:
            x1, y1, x2, y2 = map(int, line[0])
            cv2.line(line_image, (x1, y1), (x2, y2), (0, 255, 0), 2)
    else:
        debug_print("No lines detected using LSD.")
else:
    debug_print("Invalid line detection method selected. Defaulting to HoughLinesP.")
    lines = cv2.HoughLinesP(edges, rho=1, theta=np.pi/180, threshold=50, minLineLength=50, maxLineGap=10)
    if lines is not None:
        debug_print(f"HoughLinesP detected {len(lines)} lines.")
        for line in lines:
            x1, y1, x2, y2 = line[0]
            cv2.line(line_image, (x1, y1), (x2, y2), (0, 0, 255), 2)

show_image(line_image, 'Detected Lines on Pitch')

## Conclusion

The cells above show each individual step. Now we define a complete pipeline function that runs through all four steps using selectable options, and then we iterate over all 16 combinations so you can compare their final results.

In [ ]:
def process_pipeline(image, mask_method, thresh_method, edge_method, line_method):
    # --- Step 1: Crowd Masking ---
    if mask_method == 'HSV':
        hsv = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
        lower_green = np.array([25, 20, 20])
        upper_green = np.array([95, 255, 255])
        pitch_mask = cv2.inRange(hsv, lower_green, upper_green)
    elif mask_method == 'LAB':
        lab = cv2.cvtColor(image, cv2.COLOR_BGR2LAB)
        lower_lab = np.array([0, 110, 0])
        upper_lab = np.array([255, 140, 255])
        pitch_mask = cv2.inRange(lab, lower_lab, upper_lab)
    else:
        pitch_mask = None

    # Morphological operations
    kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (7,7))
    pitch_mask = cv2.morphologyEx(pitch_mask, cv2.MORPH_CLOSE, kernel, iterations=2)
    pitch_mask = cv2.dilate(pitch_mask, kernel, iterations=1)
    pitch_only = cv2.bitwise_and(image, image, mask=pitch_mask)

    # --- Step 2: White Line Extraction ---
    gray = cv2.cvtColor(pitch_only, cv2.COLOR_BGR2GRAY)
    if thresh_method == 'simple':
        ret, thresh = cv2.threshold(gray, 200, 255, cv2.THRESH_BINARY)
    elif thresh_method == 'adaptive':
        thresh = cv2.adaptiveThreshold(gray, 255, cv2.ADAPTIVE_THRESH_MEAN_C,
                                       cv2.THRESH_BINARY, blockSize=11, C=2)
    else:
        thresh = gray

    # --- Step 3: Edge Detection ---
    if edge_method == 'canny':
        edges = cv2.Canny(thresh, 50, 150, apertureSize=3)
    elif edge_method == 'sobel':
        sobelx = cv2.Sobel(thresh, cv2.CV_64F, 1, 0, ksize=3)
        sobely = cv2.Sobel(thresh, cv2.CV_64F, 0, 1, ksize=3)
        edges = cv2.magnitude(sobelx, sobely).astype(np.uint8)
    else:
        edges = thresh

    # --- Step 4: Line Detection ---
    output_image = pitch_only.copy()
    if line_method == 'hough':
        lines = cv2.HoughLinesP(edges, 1, np.pi/180, threshold=50, minLineLength=50, maxLineGap=10)
        if lines is not None:
            for line in lines:
                x1, y1, x2, y2 = line[0]
                cv2.line(output_image, (x1, y1), (x2, y2), (0,0,255), 2)
    elif line_method == 'lsd':
        lsd = cv2.createLineSegmentDetector(0)
        lines_lsd = lsd.detect(edges)[0]
        if lines_lsd is not None:
            for line in lines_lsd:
                x1, y1, x2, y2 = map(int, line[0])
                cv2.line(output_image, (x1, y1), (x2, y2), (0,255,0), 2)
    return output_image

In [ ]:
# Now, define the options for each step
mask_options = ['HSV', 'LAB']
thresh_options = ['simple', 'adaptive']
edge_options = ['canny', 'sobel']
line_options = ['hough', 'lsd']

combinations = list(product(mask_options, thresh_options, edge_options, line_options))
debug_print(f"Total combinations: {len(combinations)}")

results = []
for (mask_method, thresh_method, edge_method, line_method) in combinations:
    label = f"M:{mask_method} T:{thresh_method} E:{edge_method} L:{line_method}"
    debug_print(f"Processing combination: {label}")
    result_img = process_pipeline(image, mask_method, thresh_method, edge_method, line_method)
    results.append((label, result_img))

# Display the results in a grid
num_results = len(results)
cols = 4
rows = (num_results + cols - 1) // cols

plt.figure(figsize=(20, 5 * rows))
for i, (label, img) in enumerate(results):
    plt.subplot(rows, cols, i + 1)
    if len(img.shape) == 3:
        img_disp = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    else:
        img_disp = img
    plt.imshow(img_disp)
    plt.title(label)
    plt.axis('off')
plt.tight_layout()
plt.show()
debug_print("Completed processing all combinations.")